In [5]:
import pickle
from sklearn.tree import export_text
import pandas as pd
import numpy as np

In [6]:
filename = 'model_n.pkl'
with open(filename, 'rb') as file:
    clf = pickle.load(file)

In [51]:
for name, value in zip(clf.features, clf.feature_importances_):
  print(f'{name}: {value}')

Month: 0.03196615007303027
Age: 0.031236847087825433
Occupation: 0.024377562539120764
Annual_Income: 0.0354540924535233
Monthly_Inhand_Salary: 0.03507783978295925
Num_Bank_Accounts: 0.029126864144203225
Num_Credit_Card: 0.039292534119632586
Interest_Rate: 0.08570579350126804
Num_of_Loan: 0.02165301999750889
Delay_from_due_date: 0.05129154854029239
Num_of_Delayed_Payment: 0.038045765380553445
Changed_Credit_Limit: 0.04880619459499162
Num_Credit_Inquiries: 0.04208486519598116
Credit_Mix: 0.0597433255113575
Outstanding_Debt: 0.10844071165959701
Credit_Utilization_Ratio: 0.03767380623689618
Credit_History_Age: 0.055332259942104056
Payment_of_Min_Amount: 0.02286384967887767
Total_EMI_per_month: 0.03562940383871763
Amount_invested_monthly: 0.03783467366430675
Payment_Behaviour: 0.019612652117719214
Monthly_Balance: 0.037946520081144235
Auto Loan: 0.007825350651444334
Payday Loan: 0.008094494060825016
Credit-Builder Loan: 0.007992685953796923
Not Specified: 0.007704821282389352
Mortgage Loan:

In [8]:
df = pd.read_csv("./clean_train.csv")
X = df[clf.features]
X = np.array(X)

In [57]:
def explain(clf, X_test):
    proba = clf.predict_proba(X_test)
    classes = clf.predict(X_test)
    node_indicator, node_start = clf.decision_path(X_test)
    leaf_id = clf.apply(X_test)[0]
    

    for sample_id in range(len(X_test)):
        print(f"Sample {sample_id}:")
        print(f"Predict proba:")
        for i, c in enumerate(clf.labels):
            print(f"{c}: {proba[sample_id][i]}")
        output_tree_index = [i for i in range(clf.n_estimators) if classes[sample_id] == clf.estimators_[i].predict(X[sample_id:sample_id+1])[0]]
        tree_index = output_tree_index[0]
        n_nodes = clf.estimators_[tree_index].tree_.node_count
        feature = clf.estimators_[tree_index].tree_.feature
        threshold = clf.estimators_[tree_index].tree_.threshold

        node_index = node_indicator.indices[
            node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
        ]
        if tree_index != clf.n_estimators-1:
            node_index = [x for x in node_index if x >= node_start[tree_index] and x < node_start[tree_index+1]]
        else:
            node_index = [x for x in node_index if x >= node_start[tree_index]]

        print(f'Sample {sample_id} is {clf.labels[classes[sample_id]]}')
        print(f"Decision path of a {clf.labels[classes[sample_id]]} class tree:")
        for node_id in node_index:
            node_id = node_id - node_start[tree_index]
            # continue to the next node if it is a leaf node
            if leaf_id[sample_id] == node_id:
                continue

            # check if value of the split feature for sample 0 is below threshold
            if X_test[sample_id, feature[node_id]] <= threshold[node_id]:
                threshold_sign = "<="
            else:
                threshold_sign = ">"
            print(
                "Decision node {node} : (X_test[{sample}, {feature}] = {value}) "
                "{inequality} {threshold})".format(
                    node=node_id,
                    sample=sample_id,
                    feature=clf.features[feature[node_id]],
                    value=X_test[sample_id, feature[node_id]],
                    inequality=threshold_sign,
                    threshold=threshold[node_id],
                )
            )
        print()
        

In [58]:
explain(clf, X[10:12])

Sample 0:
Predict proba:
Poor: 0.03
Standard: 0.78
Good: 0.19
Sample 0 is Standard
Decision path of a Standard class tree:
Decision node 0 : (X_test[0, Num_of_Loan] = 1.0) <= 4.5)
Decision node 1 : (X_test[0, Num_Credit_Card] = 4.0) > 2.5)
Decision node 811 : (X_test[0, Credit_Mix] = 3.0) > 1.5)
Decision node 5937 : (X_test[0, Credit_Mix] = 3.0) > 2.5)
Decision node 11425 : (X_test[0, Outstanding_Debt] = 605.03) <= 1485.9549560546875)
Decision node 11426 : (X_test[0, Interest_Rate] = 6.0) <= 12.5)
Decision node 11427 : (X_test[0, Interest_Rate] = 6.0) > 4.5)
Decision node 11965 : (X_test[0, Payment_of_Min_Amount] = 0.0) <= 0.5)
Decision node 11966 : (X_test[0, Num_Bank_Accounts] = 2.0) <= 4.5)
Decision node 11967 : (X_test[0, Personal Loan] = 0.0) <= 1.5)
Decision node 11968 : (X_test[0, Credit_History_Age] = 321.0) > 264.5)
Decision node 12196 : (X_test[0, Monthly_Balance] = 466.46647639764313) > 32.11900520324707)
Decision node 12202 : (X_test[0, Age] = 28.0) > 21.5)
Decision node 12